### F1 Data Analysis

This is an exploratory project, scraping F1 data and analyzing.

In [10]:
# Packages reminder
    #'pip list' in terminal below to check packages installed
    #'pip install [package_name]' to to install if not

import pandas as pd
import requests
from bs4 import BeautifulSoup

#for hopefully, eventually writing to postgresql database
#some test chunks of code are still in online jupyter notebook
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

In [12]:
# Components of url that change for each race
number = '1143'
#race 1 ='1141'
place = 'australia'
# race 1 = 'bahrain'
race = 'practice-2'

base_url = 'https://www.formula1.com/en/results.html/2023/races/'

url = base_url + number + '/' + place + '/' + race + '.html'

In [13]:
# Beautiful soup preparation
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

In [14]:
# Get HTML table
tables = pd.read_html(url)
table1 = tables[0]

# Select columns wanted
table2 = table1.iloc[:,1:8]

# Create new field for driver abbrevation
table2['Abbrev'] = table2['Driver'].str[-3:]

# Remove driver abbreviation from Driver name
table2['Driver'] = table2['Driver'].str[:-3]

In [15]:
# Extract Title
title = str(soup.title.text)

# Create list split by -. Put in to dataframe with 2 rows
li = list(title.split("-"))
df = pd.DataFrame(li, columns = ['Names'])
df['Names'] = df['Names'].str.strip()

# Create fields to append to base table
Race = df.iloc[0]['Names']
Round = df.iloc[1]['Names']

# Extract date and save as variable to append to base table
date = soup.find("span", {"class": "full-date"})
date = str(date.text)

# Extract track and save as variable to append to base table
track = soup.find("span", {"class": "circuit-info"})
track = str(track.text)

In [16]:
# Append fields not part of base table and create final data frame
F1_Statisitcs = table2.assign(Race=Race, Round=Round, Date = date, Track = track)

In [17]:
# Split Time column in to minutes and hours
F1_Statisitcs["Ind_Hours"] = F1_Statisitcs['Time'].str.split(":").str[0]
F1_Statisitcs["Ind_Minutes"] = F1_Statisitcs['Time'].str.split(":").str[1]
F1_Statisitcs["Ind_Hours"] = pd.to_numeric(F1_Statisitcs["Ind_Hours"])
F1_Statisitcs["Ind_Minutes"] = pd.to_numeric(F1_Statisitcs["Ind_Minutes"])

# Create Total Min column
F1_Statisitcs["Total_Min"] = (F1_Statisitcs["Ind_Hours"]*60) + F1_Statisitcs["Ind_Minutes"]
F1_Statisitcs["Total_Min"] = pd.to_numeric(F1_Statisitcs["Total_Min"])

# Create cleaned up Gap(seconds) column. Replace nulls with 0
F1_Statisitcs['Gap_Seconds'] = F1_Statisitcs['Gap'].str.strip("+")
F1_Statisitcs['Gap_Seconds'] = F1_Statisitcs['Gap'].str.strip("s")
F1_Statisitcs['Gap_Seconds'] = F1_Statisitcs['Gap_Seconds'].fillna(0)
F1_Statisitcs['Gap_Seconds'] = pd.to_numeric(F1_Statisitcs['Gap_Seconds'])

In [18]:
# Reorder Columns
cols = ['No','Driver','Abbrev','Car','Race','Date','Track','Round','Laps','Pos','Time','Total_Min','Gap','Gap_Seconds','Ind_Hours','Ind_Minutes']
F1_Statisitcs = F1_Statisitcs[cols]

# Remove last two columns
F1_Statisitcs = F1_Statisitcs.iloc[:,:-2]

In [19]:
# Write to excel
F1_Statisitcs.to_excel('F13.xlsx', index=False)